In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install pymorphy2

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 7.1MB 11.2MB/s 


In [0]:
import re
import os
from termcolor import colored
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

path = '/content/drive/My Drive/Colab Notebooks/диплом/'

Файлы списков санскритских слов, корпуса русских слов и иностранных слов

In [0]:
#список Серенсена

with open(path + '9460-osnov-sanskritskikh-slov.txt', 'r', encoding = 'utf-8') as r:
    lines = r.readlines()
sans = [x.lower() for x in lines]
sans = [re.sub('\n', '', x) for x in sans]

In [0]:
#список из 354 тысяч санскритизмов

with open(path + '384000.txt', 'r', encoding = 'utf-8') as n:
    sans_dict = n.readlines()
    
sans_dict = [re.sub('\n','',x) for x in sans_dict]
sans_dict = set(sans_dict)
sans_dict = [x for x in sans_dict if len(x) > 2]
len(sans_dict)

353973

In [0]:
#список иностранных слов

with open(path + 'foreign_words.txt', 'r', encoding = 'utf-8') as n:
    d = n.readlines()
forn = [re.sub('\n', '', x) for x in d]
forn = [x for x in forn if len(x) > 1]
for i in ['анил', 'анупа', 'ануп', 'крошу']:
  forn.append(i)

In [0]:
#AttributeError: 'NoneType' object has no attribute 'group' - это ок

with open(path + 'dict.opcorpora.txt', 'r', encoding = 'utf-8') as n:
    rus_words = []
    lem_rus_words = []
    d = n.read()
    d = re.sub('\n', ' ', d)
    d = re.sub('\t', ' ', d)
    d = re.sub('(^|\s)[0-9]+', '||', d)
    parads = re.findall('\|\|[^\|]+', d)
    for par in parads:
      try:
        lem = re.search('\|\|\s([А-Яа-яёЁ]+)', par).group(1)
        if 'Ё' in lem:
            lem = re.sub('Ё', 'Е', lem)
            
        if lem.lower() not in forn + sans:
            lem_rus_words.append(lem.lower())
            words = re.findall('[А-Яа-яЁё]+', par)
            for w in words:
                if 'Ё' in w:
                    w = re.sub('Ё', 'Е', w)
                rus_words.append(w.lower())
      except AttributeError:
        continue

In [0]:
rus_words.remove('даму')
rus_words.remove('дама')
rus_words.remove('кишку')
rus_words.remove('пилу')
rus_words.remove('руру')
rus_words.remove('турья')

cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
for case in cases:
  list(rus_words).append(morph.parse('акт')[0].inflect({case}).word)

rus_words = [x for x in rus_words if len(x) > 1]
rus_words.append('и')
rus_words = set(rus_words)

In [0]:
lem_rus_words = set(lem_rus_words)

In [0]:
sans_dict = [z for z in sans_dict if z not in lem_rus_words]

Разделение санскритизмов на 3 группы (по окончанию)

In [0]:
indecl = []
decl = []
stem = []

vows = ['а', 'я', 'ы']

for i in sans + sans_dict:
    if i.endswith('и') or i.endswith('у') or i.endswith('ю') or i.endswith('е') or i.endswith('о'):
        indecl.append(i)
    elif i[-1:] in vows:
        decl.append((i[:-1], i))
    else:
        stem.append(i)

indecl = set(indecl)
stem = set(stem)

In [0]:
#exep - список исключений

preps = ['в', 'без', 'до', 'из', 'к', 'на', 'по', 'о', 'от', 'перед', 'при', 'через', 'с', 'у', 'за', 'над', 'об', 'под', 'про', 'для', 'ко', 'обо', 'ото', 'во', 'безо', 'передо', 'со', 'надо']
prons = ['меня', 'нем', 'наш', 'наша', 'ваш', 'она', 'они', 'кто', 'что', 'ними', 'ней', 'наши', 'ваши', 'вашем', 'тот', 'та', 'те', 'той', 'нас', 'ней', 'нам', 'них', 'вас', 'вам']
conjs = ['или', 'но', 'да', 'либо', 'ради', 'нет', 'да']
advs = ['так', 'туда', 'как', 'тут', 'там', 'сами', 'сам', 'сама']

exep = preps + prons + conjs + advs

indecl = [x for x in indecl if x not in exep]

In [0]:
decl_d = {}
stem_d = {}

for i in list(set([x[0][:2] for x in decl])):
  decl_d[i] = []

for i in list(set([x[:2] for x in stem])):
  stem_d[i] = []

for i in decl:
  decl_d[i[0][:2]].append(i)

for i in stem:
  stem_d[i[:2]].append(i)

Чтение файла с корпусом

In [0]:
with open(path + 'aranyakaparva_corpus2.txt', 'r', encoding = 'utf-8') as r:
  corpus = r.read()

In [0]:
parts = re.findall('(\[[0-9]+-[0-9]+\]|\[[0-9]+\])\s+<\!\s--\ssanskrit\s-->\s+([^+^<]*)\s+<\!\s--\srus\s-->\s+([^+^<^\[]*)', corpus)

In [0]:
len(parts)

2033

Файлы соответствий санскритской латиницы кириллице

In [0]:
with open(path + 'translation.txt', 'r', encoding = 'utf-8') as t:
    trans = t.readlines()

In [0]:
with open(path + 'correct_trans.txt', 'r', encoding = 'utf-8') as t:
    c = t.readlines()

In [0]:
d = {}
d_combs = {}

for i in trans:
    if re.search('Replace\s"(.*?)","(.*?)"', i):
        sans = re.search('Replace\s"(.*?)","(.*?)"', i).group(1)
        rus = re.search('Replace\s"(.*?)","(.*?)"', i).group(2)
        if len(rus) > 1:
            d_combs[sans] = rus
        else:
            d[sans] = rus

corr = {}

for i in c:
    if re.search('Replace\s"(.*?)","(.*?)"', i):
        bad = re.search('Replace\s"(.*?)","(.*?)"', i).group(1)
        good = re.search('Replace\s"(.*?)","(.*?)"', i).group(2)
        corr[bad] = good 

Функция перевода санскритской латиницы в кириллицу

In [0]:
def translate(s):
    string = []
    for word in s.split():
        trans_word = []
        for k, v in d_combs.items():
            if k in word:
              word = re.sub(k, d_combs[k], word)
            
        for letter in word:
            if letter in d:
                trans_word.append(d[letter])
            else:
                trans_word.append(letter)
        trans_word = ''.join(trans_word)
        # print(trans_word)
        vs = ['а', 'я', 'ю', 'е', 'о', 'ы', 'и', 'у']
        for k, v in corr.items():
            if k in trans_word:
              # print(k)
              # print(v)
              # #чтобы мягкий знак не удалялся между "р" и гласной (сурья, а не суря)
              # check = False
              # if k == 'ь':
              #   arr = [i for i in trans_word]
              #   for indx, i in enumerate(arr):
              #     if i == 'р' and len(arr) - indx >= 3 and arr[indx+1] == 'ь' and arr[indx+2] in vs:
              #       check = True
              # elif check == False and k == 'ь':
              #   trans_word = re.sub(k, corr[k], trans_word)
              # else:
              trans_word = re.sub(k, corr[k], trans_word)
        trans_word = [re.sub('॥', '', x) for x in trans_word]
        trans_word = [re.sub('[0-9]+', '', x) for x in trans_word]
        trans_word = [re.sub('\s+', '', x) for x in trans_word]
        trans_word = ''.join(trans_word)
        string.append(trans_word)
    return string

In [0]:
a = [i for i in 'курья']
vs = ['а', 'я', 'ю', 'е', 'о', 'ы', 'и', 'у']
for indx, i in enumerate(a):
  if i == 'р' and len(a) - indx >= 3 and a[indx+1] == 'ь' and a[indx+2] in vs:
    print(1)

1


In [0]:
print(translate('janamejaya'))
print(translate('vaiśampāyana'))
print(translate("sūryasyaiva"))

['джанамеджая']
['вайшампаяна']
['сурясяйва']


Поиск соответствий

In [0]:
len(parts)

2033

In [0]:
parts[21:28]

[('[1-4]',
  "vaiśampāyana uvāca ।\nśaunakenaivamuktastu kuntīputro yudhiṣṭhiraḥ ।\npurohitamupāgamya bhrātṛmadhye'bravīdidam ॥1\nprasthitaṃ mānuyāntīme brāhmaṇā vedapāragāḥ ।\nna cāsmi pālane śakto bahuduḥkhasamanvitaḥ ॥2\nparityaktuṃ na śaknomi dānaśaktiśca nāsti me ।\nkathamatra mayā kāryaṃ bhagavāṃstadbravītu me ॥3\nmuhūrtamiva sa dhyātvā dharmeṇānviṣya tāṃ gatim ।\nyudhiṣṭhiramuvācedaṃ dhaumyo dharmabhṛtāṃ varaḥ ॥4",
  'Вайшампаяна сказал:\nВыслушав речь Шаунаки, Юдхиштхира, сын Кунти, сопровождаемый братьями, пошел к своему родовому жрецу и сказал ему: «Не следовало бы этим брахманам, усвоившим (учение) Вед, сопровождать меня в моем изгнании. Многие беды постигли меня, и я не в силах обеспечить им защиту. И оставить их не могу, и даяниями поддерживать не способен. Как мне тут быть, да разъяснит почтенный!» На какое-то время Дхаумья, достойнейший из блюстителей закона, предался размышлениям, дабы найти путь, согласный с дхармой, а потом отвечал Юдхиштхире:\n'),
 ('[5-12]',
  "purā

In [0]:
[x[1] for x in parts[11:15]]

['brāhmaṇā ūcuḥ ।\nasmatpoṣaṇajā cintā mā bhūtte hṛdi pārthiva ।\nsvayamāhṛtya vanyāni anuyāsyāmahe vayam ॥10\nanudhyānena japyena vidhāsyāmaḥ śivaṃ tava ।\nkathābhiścānukūlābhiḥ saha raṃsyāmahe vane ॥11',
 "yudhiṣṭhira uvāca ।\nevametanna sandeho rameyaṃ brāhmaṇaiḥ saha ।\nnyūnabhāvāttu paśyāmi pratyādeśamivātmanaḥ ॥12\nkathaṃ drakṣyāmi vaḥ sarvānsvayamāhṛtabhojanān ।\nmadbhaktyā kliśyato'narhāndhikpāpāndhṛtarāṣṭrajān ॥13",
 'vaiśampāyana uvāca ।\nityuktvā sa nṛpaḥ śocanniṣasāda mahītale ।\ntamadhyātmaratirvidvāñśaunako nāma vai dvijaḥ ।\nyoge sāṅkhye ca kuśalo rājānamidamabravīt ॥14\nśokasthānasahasrāṇi bhayasthānaśatāni ca ।\ndivase divase mūḍhamāviśanti na paṇḍitam ॥15\nna hi jñānaviruddheṣu bahudoṣeṣu karmasu ।\nśreyoghātiṣu sajjante buddhimanto bhavadvidhāḥ ॥16\naṣṭāṅgāṃ buddhimāhuryāṃ sarvāśreyovighātinīm ।\nśrutismṛtisamāyuktāṃ sā rājaṃstvayyavasthitā ॥17\narthakṛcchreṣu durgeṣu vyāpatsu svajanasya ca ।\nśārīramānasairduḥkhairna sīdanti bhavadvidhāḥ ॥18\nśrūyatāṃ cābhidhāsyāmi 

In [0]:
def search(corpus):

  res = []
  count = 0

  reg_space = re.compile('\s{2,}', re.DOTALL)
  reg_punct = re.compile('[^\w\s]', re.DOTALL)
  found = []
  for part in parts[21:28]:
    count += 1
    print(count)
    number = part[0]
    rus_part = part[2]
    sans_part = part[1]

    upper_words = []
    #found = []
    
    #так только 1776 нашлось
    # for indx, word in enumerate(sans_part.split()):
    #   cyr_sans = translate(word)
    #   for idx, rus in enumerate(rus_part.split()):
    #     if re.search('[а-яА-Я]+', cyr_sans[0]) and len(cyr_sans[0]) > 3 and rus.startswith(cyr_sans[0][:-1]):
    #       found.append((number, [indx, word], [idx, rus]))

    vs = ['а', 'я', 'ю', 'е', 'о', 'ы', 'и', 'у']

    # text = rus_part

    text = re.sub(r'\d', '', rus_part)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[A-Za-z]', '', text)

    if re.search(reg_punct, text):
        text = re.sub(reg_punct, ' ', text)
      
    if re.search(reg_space, text):
        text = re.sub(reg_space, ' ', text)

    for indx, word in enumerate(text.split()):
        f = False

        # word = re.sub(r'\d', '', word)
        # word = re.sub(r'\n', ' ', word)
        # word = re.sub(r'[A-Za-z]', '', word)

        # if re.search(reg_punct, word):
        #     word = re.sub(reg_punct, '', word)

        # if word == '':
        #   continue

        if word[0].isupper() == True and indx > 0:
          upper_words.append((word.lower(), indx))
        word = word.lower()
        if word in indecl and word not in rus_words and word not in exep:
            found.append((word, indx))
          
        #список склоняемых санскритизмов, заканчивающихся на гласную
        elif word[:2] in decl_d:
          if word in [x[1] for x in decl_d[word[:2]]] and word not in exep and word not in rus_words:
              found.append((word, indx))
              f = True
          else:
            for i in decl_d[word[:2]]:
                s = i[0]
                if word.startswith(s) and word[-1] in vs and len(word) - len(s) == 1 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                    f = True
                elif len(word) > 2 and word.startswith(s) and word[-3] in vs and word.endswith('ми') and len(word) - len(s) == 3 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                    f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('м') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                    f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('в') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                    f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('й') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                    f = True
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('х') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                    f = True

        if f == False and word[:2] in stem_d:
          if word in [x for x in stem_d[word[:2]]] and word not in exep and word not in rus_words:
            found.append((word, indx))
          else:
            for s in stem_d[word[:2]]:
                if word.startswith(s) and word[-1] != 'ь' and len(word) - len(s) <= 1 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                elif len(word) > 2 and word.startswith(s) and word[-3] in vs and word.endswith('ми') and len(word) - len(s) == 3 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('м') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('в') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('й') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))
                elif len(word) > 1 and word.startswith(s) and word[-2] in vs and word.endswith('х') and len(word) - len(s) == 2 and word not in exep and word not in rus_words:
                    found.append((word, indx))

    for i in upper_words:
      word = i[0]
      for s in sans:
          if word.startswith(s) and word[-1] != 'ь' and len(word) - len(s) <= 1 and word not in exep:
              found.append((word, indx))
          elif word.startswith(s) and word.endswith('ми') and len(word) - len(s) == 3 and word not in exep:
              found.append((word, indx))
          elif word.startswith(s) and word.endswith('м') and len(word) - len(s) == 2 and word not in exep:
              found.append((word, indx))
          elif word.startswith(s) and word.endswith('в') and len(word) - len(s) == 2 and word not in exep:
              found.append((word, indx))
          elif word.startswith(s) and word.endswith('й') and len(word) - len(s) == 2 and word not in exep:
              found.append((word, indx))
          elif word.startswith(s) and word.endswith('х') and len(word) - len(s) == 2 and word not in exep:
              found.append((word, indx))

    indxs = set()
    indxr = set()

    # text = sans_part

    text = re.sub(r'\d', '', sans_part)
    if "'" in text:
      text = re.sub("'", " a", text)
    text = re.sub(r'\n', ' ', text)

    if re.search(reg_punct, text):
        text = re.sub(reg_punct, ' ', text)
      
    if re.search(reg_space, text):
        text = re.sub(reg_space, ' ', text)

    for indx, word in enumerate(text.split()):

      # word = re.sub(r'\d', '', word)
      # if "'" in word:
      #   word = re.sub("'", " a", word)
      # word = re.sub(r'\n', ' ', word)

      # if re.search(reg_punct, word):
      #     word = re.sub(reg_punct, '', word)

      # if word == '':
      #   continue

      def proc_short(sans, rus, flex):
        s = translate(sans)[0]
        if s.startswith(rus[:-flex]):
          if len(sans) - len(rus) < 4:
            res = sans
          else:
            t = sans[:len(rus)]
            if translate(t[-1])[0] == s[:len(rus)][-1]:
              res = t
            else:
              t = sans[:len(rus) + 1]
              res = t
          return res

      def proc_long(sans, rus):
        s = translate(sans)[0]
        if rus[:-1] in s:
          start = s.find(rus[:-1])
          end = start + len(rus)
          if s[start:end] == rus:
            cut = sans[start:end]
          elif s[start:end][:-1] == rus[:-1]:
            cut = sans[start:end]
          elif s[start+1:end+1] == rus:
            cut = sans[start+1:end+1]
          elif s[start-1:end-1] == rus:
            cut = sans[start-1:end-1]
          else:
            cut = sans[start:end]
          return cut


      if re.search('[a-z]', word):
        rus_sans = translate(word)
        # print(word)
        # print(rus_sans)
        for i in found:
          s = i[0]
          if 'ь' in s:
            s = re.sub('ь', '', s)
          if len(indxs) > 0 and (indx, word) not in indxs and (i[1], i[0]) not in indxr and rus_sans[0].startswith(s[:-1]):
            part = proc_short(word, s, 1)
            # print('PART' + part)
            res.append((number, [indx, part], [i[1], i[0]]))
            indxs.add((indx, word))
            indxr.add((i[1], i[0]))
            print((number, [indx, part], [i[1], i[0]]))
          elif i[0].endswith('й') or i[0].endswith('м') or i[0].endswith('в') or i[0].endswith('х'):
            if len(indxs) > 0 and (indx, word) not in indxs and (i[1], i[0]) not in indxr and rus_sans[0].startswith(s[:-2]):
              part = proc_short(word, s, 2)
              res.append((number, [indx, part], [i[1], i[0]]))
              indxs.add((indx, word))
              indxr.add((i[1], i[0]))
              print((number, [indx, part], [i[1], i[0]]))
            elif len(indxs) == 0 and rus_sans[0].startswith(s[:-2]):
              part = proc_short(word, s, 2)
              res.append((number, [indx, part], [i[1], i[0]]))
              indxs.add((indx, word))
              indxr.add((i[1], i[0]))
              print((number, [indx, part], [i[1], i[0]]))
          elif i[0].endswith('ми'):
            if len(indxs) > 0 and (indx, word) not in indxs and (i[1], i[0]) not in indxr and rus_sans[0].startswith(s[:-3]):
              part = proc_short(word, s, 3)
              res.append((number, [indx, part], [i[1], i[0]]))
              indxs.add((indx, word))
              indxr.add((i[1], i[0]))
              print((number, [indx, part], [i[1], i[0]]))
            elif len(indxs) == 0 and rus_sans[0].startswith(s[:-3]):
              part = proc_short(word, s, 3)
              res.append((number, [indx, part], [i[1], i[0]]))
              indxs.add((indx, word))
              indxr.add((i[1], i[0]))
              print((number, [indx, part], [i[1], i[0]]))
          #чтоб в "сурапитриганаякшасевитан" находились "ганы" и "якши"
          elif len(indxs) > 0 and (indx, word) not in indxs and (i[1], i[0]) not in indxr and s[:-1] in rus_sans[0]:
              part = proc_long(word, s)
              res.append((number, [indx, part], [i[1], i[0]]))
              indxs.add((indx, word))
              indxr.add((i[1], i[0]))
              print('new' + str((number, [indx, part], [i[1], i[0]])))
          elif len(indxs) == 0 and rus_sans[0].startswith(s[:-1]):
            part = proc_short(word, s, 1)
            res.append((number, [indx, part], [i[1], i[0]]))
            indxs.add((indx, word))
            indxr.add((i[1], i[0]))
            print((number, [indx, part], [i[1], i[0]]))

  return res, found

In [0]:
%%time
res, found = search(parts)

1
('[1-4]', [0, 'vaiśampāyana'], [0, 'вайшампаяна'])
('[1-4]', [2, 'śaunake'], [4, 'шаунаки'])
('[1-4]', [3, 'kuntī'], [7, 'кунти'])
('[1-4]', [4, 'yudhiṣṭhiraḥ'], [5, 'юдхиштхира'])
('[1-4]', [10, 'brāhmaṇā'], [22, 'брахманам'])
('[1-4]', [31, 'dharmeṇ'], [76, 'дхармой'])
('[1-4]', [34, 'yudhiṣṭhira'], [80, 'юдхиштхире'])
('[1-4]', [35, 'dhaumyo'], [64, 'дхаумья'])
2
('[5-12]', [9, 'savitā'], [11, 'савитар'])
new('[5-12]', [21, 'tejaḥ'], [53, 'теджас'])
('[5-12]', [53, 'bhīmena'], [136, 'бхима'])
('[5-12]', [54, 'kārtavīryeṇa'], [137, 'картавирья'])
('[5-12]', [55, 'vainyena'], [138, 'вайнья'])
('[5-12]', [56, 'nahuṣeṇa'], [140, 'нахуша'])
new('[5-12]', [58, 'samādhi'], [148, 'самадхи'])
('[5-12]', [63, 'dharmāt'], [76, 'дхармой'])
('[5-12]', [68, 'dharmeṇa'], [161, 'дхарме'])
3
('[13-14]', [1, 'dhaumyena'], [64, 'дхаумья'])
('[13-14]', [4, 'dharmarājo'], [76, 'дхармой'])
('[13-14]', [11, 'dharmātmā'], [161, 'дхарме'])
('[13-14]', [16, 'prāṇāyāmena'], [45, 'пранаяму'])
4
('[15]', [0, 

Длинные слова, после того, как в них нашлось какое-то имя, не должны отбрасываться, а обрабатываться дальше, потому что в них могут быть еще другие имена. - ДОДЕЛАТЬ

In [0]:
#'юдхиштхирамувачедан'
# sans = 'yudhiṣṭhiramuvācedaṃ'
# rus = 'юдхиштхире'
sans = 'śaunakenaivamuktastu'
rus = 'шаунаки'
s = translate(sans)[0]

if s.startswith(rus[:-1]):
  if len(sans) - len(rus) < 4:
    print(sans)
  else:
    t = sans[:len(rus)]
    if translate(t[-1])[0] == s[:len(rus)][-1]:
      print(t)
    # elif translate(t[-1])[0] == s[:len(rus)][-1]:
    #   print(t)
    else:
      t = sans[:len(rus) + 1]
      if translate(t[-1])[0] == s[:len(rus)][-1]:
        print(t)
      else:
        print('else' + t)

śaunake


In [0]:
sans = 'dhanvantarirdhūmaketurādidevo'
rus = 'дханвантари'
s = translate(sans)[0]

if rus[:-1] in s:
  start = s.find(rus[:-1])
  end = start + len(rus)
  if s[start:end] == rus:
    cut = sans[start:end]
    print(cut)
  elif s[start:end][:-1] == rus[:-1]:
    cut = sans[start:end]
    print(cut)
  elif s[start+1:end+1] == rus:
    cut = sans[start:end]
    print(cut)
  elif s[start-1:end-1] == rus:
    cut = sans[start:end]
    print(cut)
  else:
    print('else' + cut)

dhanvantari


In [0]:
len(res)

66

**Раскрашиваем санскритизмы в тексте**

In [0]:
parts[21][0]

'[1-4]'

In [0]:
[x for x in res if x[0] == parts[21][0]]

[('[1-4]', [0, 'vaiśampāyana'], [0, 'вайшампаяна']),
 ('[1-4]', [2, 'śaunake'], [4, 'шаунаки']),
 ('[1-4]', [3, 'kuntī'], [7, 'кунти']),
 ('[1-4]', [4, 'yudhiṣṭhiraḥ'], [5, 'юдхиштхира']),
 ('[1-4]', [10, 'brāhmaṇā'], [22, 'брахманам']),
 ('[1-4]', [31, 'dharmeṇ'], [76, 'дхармой']),
 ('[1-4]', [34, 'yudhiṣṭhira'], [80, 'юдхиштхире']),
 ('[1-4]', [35, 'dhaumyo'], [64, 'дхаумья'])]

In [0]:
parts[21][1]

"vaiśampāyana uvāca ।\nśaunakenaivamuktastu kuntīputro yudhiṣṭhiraḥ ।\npurohitamupāgamya bhrātṛmadhye'bravīdidam ॥1\nprasthitaṃ mānuyāntīme brāhmaṇā vedapāragāḥ ।\nna cāsmi pālane śakto bahuduḥkhasamanvitaḥ ॥2\nparityaktuṃ na śaknomi dānaśaktiśca nāsti me ।\nkathamatra mayā kāryaṃ bhagavāṃstadbravītu me ॥3\nmuhūrtamiva sa dhyātvā dharmeṇānviṣya tāṃ gatim ।\nyudhiṣṭhiramuvācedaṃ dhaumyo dharmabhṛtāṃ varaḥ ॥4"

In [0]:
def highlight(parts, res):

    colors = ['hotpink', 'peru', 'navy', 'orangered', 'darkslateblue', 'purple', 'seagreen', 'salmon', 'khaki', 'brown', 'gold', 'maroon', 'teal', 'sandybrown', 'darkgreen', 'magenta', 'olive', 'darkred', 'rosybrown']
    
    # keywords to be colored
    x = [x for x in res if x[0] == parts[21][0]]
    kw_sans = [i[1][1] for i in x]
    kw_rus = [i[2][1] for i in x]

    rus = parts[21][2].lower()
    sans = parts[21][1].lower()

    rus_text = ''
    for word in rus.split():
      check = False
      for k, color in zip(kw_rus, colors[:len(x)+1]):
        done = word.replace(k, "<b style='color:{}'>".format(color) + k + "</b>")
        if done != word:
          check = True
          break
      if check == False:
        rus_text = rus_text + word + ' '
      else:
        rus_text = rus_text + done + ' '
      

    # rus = rus.replace(k, "<b style='color:{}'>".format(color) + k + "</b>")
    # rus = rus.replace("\n","<br>")
    # rus = rus.replace("\t", "&nbsp;&nbsp;&nbsp;&nbsp;")

    sans_text = ''
    for word in sans.split():
      check = False
      for k, color in zip(kw_sans, colors[:len(x)+1]):
        done = word.replace(k, "<b style='color:{}'>".format(color) + k + "</b>")
        if done != word:
          check = True
          break
      if check == False:
        sans_text = sans_text + word + ' '
      else:
        sans_text = sans_text + done + ' '

    #   sans = sans.replace(k, "<b style='color:{}'>".format(color) + k + "</b>")
    # sans = sans.replace("\n","<br>")
    # sans = sans.replace("\t", "&nbsp;&nbsp;&nbsp;&nbsp;")

    text = parts[21][0] + '<br><br>' + rus_text + '<br><br>' + sans_text + '<br><br>*****************************************************************'
    for i in set(re.findall('[।॥][\s0-9]+', text)):
      text = re.sub(i, i + '<br>', text)
    return text

corp = highlight(parts, res)
 
with open("paral_corp.html", "w", encoding = 'utf-8') as file:
    file.write(corp)

In [0]:
# with open("paral_corpus.txt", "w", encoding = 'utf-8') as file:
#     file.write(corp)

In [0]:
print(len(set(found)))
set(found)

83


{('адити', 132),
 ('ангарака', 29),
 ('арка', 5),
 ('арьяман', 1),
 ('асурами', 51),
 ('ашваттха', 81),
 ('брахма', 38),
 ('брахманам', 22),
 ('брахманов', 13),
 ('брахманов', 180),
 ('брихаспати', 25),
 ('будха', 27),
 ('бхага', 2),
 ('бхаскару', 62),
 ('бхима', 136),
 ('вайнья', 138),
 ('вайшампаяна', 0),
 ('вайшравана', 42),
 ('варуна', 99),
 ('вивасван', 31),
 ('вишну', 39),
 ('ганами', 43),
 ('года', 77),
 ('года', 79),
 ('двапара', 63),
 ('джанамеджая', 0),
 ('дханвантари', 128),
 ('дхарма', 173),
 ('дхарме', 161),
 ('дхармой', 76),
 ('дхармы', 54),
 ('дхаумьи', 5),
 ('дхаумьи', 28),
 ('дхаумья', 21),
 ('дхаумья', 32),
 ('дхаумья', 64),
 ('индра', 30),
 ('кала', 69),
 ('картавирья', 137),
 ('каурава', 7),
 ('каштха', 70),
 ('крита', 61),
 ('кунти', 7),
 ('махабхараты', 133),
 ('мира', 93),
 ('мухурта', 72),
 ('нарада', 18),
 ('нахуша', 140),
 ('нишачарами', 49),
 ('пакша', 73),
 ('партхе', 35),
 ('питарами', 42),
 ('пранаяму', 45),
 ('пуруша', 85),
 ('пушан', 4),
 ('рави', 7),
 (

In [0]:
print(colored('hello', 'red'), colored('world', 'green'))

In [0]:
a = [1]
a[-2]